In [91]:
import dhlab as dh
import pandas as pd
from dhlab.nbtokenizer import tokenize
import jsonlines
dh.css()

# Lemmatize a set

a = dh.totals(100000)
lemma = dh.WordLemma(list(a.index)).lemmas
reduced_set = pd.DataFrame(lemma.merge(a.reset_index(), left_on=0, right_on='index').groupby(2)['freq'].max())
lower_set = pd.DataFrame(reduced_set.groupby(lambda x: x.lower())['freq'].sum())
lower_set.sort_values(by='freq', ascending=False)

# Generate sentences

In [125]:
corpus = dh.Corpus(doctype='digibok', lang='nob', ddk='0*', from_year=1980, limit = 10000)
for i in range(1,9):
    dewey = f"{str(i)}*"
    corpus.add(dh.Corpus(doctype='digibok', lang='nob', ddk=dewey, from_year=1980, limit = 10000))

In [126]:
corpus.size

72287

In [127]:
def generate_word_sequences_from_corpus(corpus, samples, limit):
    sentences = dh.Concordance(corpus.sample(samples), query = "og OR i OR på OR av OR eller OR har OR ha OR hatt OR er OR var", limit = limit)
    a = sentences.show(style=False, n=limit).concordance.apply(lambda x:x.replace('<b>','').replace('</b>','')).values
    return a

In [96]:
def convert(sentence):
    """Tokenize i a string, turn all tokens to lower case and 
    remove punctuations and other stuff not normally part of a word"""
    lst = tokenize(sentence)
    lowered = [x.lower() for x in lst if x.isalpha()]
    return lowered

## Generate json lines

In [128]:
for i in range(5000):
    if i%100 == 0:
        print(i)
    a = generate_word_sequences_from_corpus(corpus, 100, 1000)
    with open('sentences_varied_corpus.textlines',mode='a') as wr:
        for sent in map(convert, a):
            wr.write(' '.join(sent))
            wr.write('\n')
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
